```{contents}
```
## State Persistence

**State persistence** in LangGraph is the mechanism by which the **entire execution state of a graph is stored, versioned, and recoverable across time, failures, and user interactions**.
It transforms LangGraph from a simple workflow engine into a **reliable, long-running, production-grade system**.

---

### **1. Why State Persistence Is Necessary**

LLM workflows are **not atomic**. They involve:

* multi-step reasoning
* tool calls
* human approvals
* long-running processes
* failure recovery
* multi-session conversations

Without persistence, **any crash destroys the workflow**.

| With Persistence  | Without Persistence |
| ----------------- | ------------------- |
| Survives restarts | Lost progress       |
| Supports recovery | Must restart        |
| Enables debugging | No traceability     |
| Human-in-the-loop | Impossible          |

---

### **2. What Is Persisted**

LangGraph persists **execution-level state**:

| Component          | Description                |
| ------------------ | -------------------------- |
| Global State       | The shared TypedDict state |
| Node Outputs       | Partial updates from nodes |
| Execution Position | Current node               |
| Graph Structure    | Versioned graph definition |
| Thread ID          | Workflow identity          |
| Checkpoints        | Time-ordered snapshots     |
| Metadata           | timestamps, user, config   |

---

### **3. Persistence Architecture**

```
Client Request
     ↓
LangGraph Runtime
     ↓
Checkpoint Manager
     ↓
State Store (Redis / Postgres / DynamoDB)
```

Each transition produces a **checkpoint**:

```
(State_t, Node, Metadata) → Persistent Storage
```

---

### **4. Checkpointing Model**

Every node execution creates a snapshot:

```text
Checkpoint = {
  thread_id,
  node_name,
  state,
  timestamp,
  metadata
}
```

This enables:

* replay
* rollback
* debugging
* recovery

---

### **5. Enabling Persistence in Code**

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("state.db")

graph = builder.compile(checkpointer=checkpointer)
```

Running with identity:

```python
config = {"configurable": {"thread_id": "user-123"}}
graph.invoke(input_state, config)
```

---

### **6. Recovery & Resume**

If execution crashes:

```python
graph.invoke(input_state, config)
```

LangGraph **resumes from the latest checkpoint** for that thread.

---

### **7. Long-Term Conversations**

Because state is persisted:

* conversations are **not tied to process memory**
* multiple servers can share the same workflow
* horizontal scaling is safe

---

### **8. Human-in-the-Loop Support**

Execution can pause:

```
Reason → Analyze → [WAIT FOR HUMAN] → Continue
```

After human input, the same thread resumes.

---

### **9. Storage Backends**

| Backend    | Use Case     |
| ---------- | ------------ |
| SQLite     | local dev    |
| PostgreSQL | production   |
| Redis      | high-speed   |
| DynamoDB   | serverless   |
| S3         | cold storage |

---

### **10. Production Benefits**

| Capability           | Enabled |
| -------------------- | ------- |
| Crash recovery       | ✔       |
| Auditing             | ✔       |
| Compliance           | ✔       |
| Multi-session agents | ✔       |
| Autonomous systems   | ✔       |
| Enterprise workflows | ✔       |

---

### **11. Mental Model**

> LangGraph persistence = **transaction log for intelligence**

Each step of thinking becomes a **durable fact**.

---

### **12. Summary**

State persistence converts LangGraph from:

> *a workflow runner* → **an intelligent distributed system**


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    count: int
    done: bool

def increment(state: State):
    print("Incrementing:", state["count"])
    return {"count": state["count"] + 1}

def check_done(state: State):
    done = state["count"] >= 5
    print("Done?", done)
    return {"done": done}


In [6]:
from langgraph.checkpoint.memory import InMemorySaver

from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("inc", increment)
builder.add_node("check", check_done)

builder.set_entry_point("inc")
builder.add_edge("inc", "check")

builder.add_conditional_edges(
    "check",
    lambda s: END if s["done"] else "inc",
    {"inc": "inc", END: END}
)


checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [7]:
config = {"configurable": {"thread_id": "demo"}}

graph.invoke({"count": 0, "done": False}, config)


Incrementing: 0
Done? False
Incrementing: 1
Done? False
Incrementing: 2
Done? False
Incrementing: 3
Done? False
Incrementing: 4
Done? True


{'count': 5, 'done': True}